In [1]:
#from preprocessing_functions import *
#%matplotlib inline 
import numpy as np   # generic stuff
#import matplotlib.pyplot as plt

#### REMOVE THIS LINE BEFORE SUBMISSION
# import pandas as pd
#######################################################################

from lib.proj1_helpers import * #the helper provided for the project
#from lib.costs import *

# choose which implementations you would like
from lib.implementations import *
#from implementations import * #our implementations of the functions done by us


#import datetime
#import operator
#from helpers import * #helpers of exo 2
# Useful starting lines

#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload

In [2]:
# Folder where to take the data from
DATA_FOLDER = '../data/'

# Load training and testing data
y_train, tx_train, ids_train = load_csv_data(DATA_FOLDER+'train.csv',sub_sample=False)
y_test, tx_test, ids_test = load_csv_data(DATA_FOLDER+'test.csv',sub_sample=False)

In [3]:
# Check the shapes of the data
print(tx_train.shape)
print(tx_test.shape)
print(y_train.shape)

(250000, 30)
(568238, 30)
(250000,)


In [36]:
x = np.array([[1,2,3],[4,5,6]])
print(x)
mean_x = np.mean(aaa, axis=0)
print(mean_x)
aaa = x - mean_x
print(aaa)
std_x

[[1 2 3]
 [4 5 6]]
[ 2.5  3.5  4.5]
[[-1.5 -1.5 -1.5]
 [ 1.5  1.5  1.5]]


In [4]:
def build_k_indices(y, k_fold, seed):
    """Build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [5]:
def split_jets_mask(tx):
    """Creates a mask from array tx corresponding to feature 22 (number of jets)."""
    idx_cat = 22
    return {
        0: tx[:,idx_cat] == 0,
        1: tx[:,idx_cat] == 1,
        2: tx[:,idx_cat] == 2,
        3: tx[:,idx_cat] == 3,
    }

In [6]:
def clean_missing_values(tx, mean=False):
    """Replace missing values (-999) of tx by the mean (if mean==True) or by the median (if mean==False) of the 
       column in which the missing value belongs.
       Returns the modified matrix and a matrix similar to tx but with 1 instead of -999 and 0 otherwise."""
    
    nan_values = (tx==-999)*1
    for col in range(tx.shape[1]):
        # Select entries of col that have values -999
        column = tx[:,col][tx[:,col]!=-999]
        
        # If the column col contains missing values, replace them by the median or the mean
        if len(column) != 0:
            if mean==False:
                replace_value = np.median(column)
            else:
                replace_value = np.mean(column)
            tx[:,col][tx[:,col]==-999] = replace_value
            
    return tx, nan_values

In [7]:
def keep_unique_cols(tx):
    """Input: matrix tx. If two (or more) columns of tx are equal, keep only one of them.
       If a column has all of its entries identical, do not keep it (it will be replaced by the column of 1s).
       Returns the indices of the kept columns."""

    unique_cols_ids = [0]
    for i in range(1,tx.shape[1]):
        # Check if all entries of column i are identical
        erase = np.sum((tx[:,i]==tx[0,i])*1)==len(tx[:,i]) 
        
        # If all entries are not identical, check if the column is equal to another one amongst the 
        # already chosen columns
        if erase == False: 
            id_loop = unique_cols_ids
            for j in id_loop:
                if np.sum(tx[:,i]-tx[:,j])==0:
                    erase = True
                    break
                    
        # If the column is not equal to another one, nor it has all identical entries, keep it
        if erase == False: 
            unique_cols_ids.append(i)
    
    return unique_cols_ids

In [8]:
def add_cross_prod(tx, i, j):
    """Add a column to tx correponding to the product of the entries of the i-th and j-th columns."""
    return np.concatenate((tx, np.array([tx[:,i]*tx[:,j]]).T), axis=1)

def add_all_cross_prod(tx, selected_cols=[]):
    """Add all products between 2 products"""
    if selected_cols==[]:
        selected_cols=range(tx.shape[1])
    for idx, i in enumerate(selected_cols):
        #print(idx)
        for j in selected_cols[idx+1:]:
            tx = add_cross_prod(tx, i, j)
    return tx

In [9]:
def standardize(x):
    """Standardize the data set x."""
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    for idx in range(len(std_x)):
        if std_x[idx] > 1e-15:
            x[:,idx] = x[:,idx] / std_x[idx]
    return x, mean_x, std_x

In [10]:
def add_ones(tx):
    """Add a column of 1s to matrix tx."""
    return np.concatenate((tx, np.ones([tx.shape[0],1])), axis=1)

In [33]:
def preprocess_data(tx, unique_cols=[], stdize="before"):
    #print('Cleaning features')
    tx, nan_values = clean_missing_values(tx)
    np.append(tx, nan_values[:,0]) # Add dummy variable to keep the information saying when the mass is -999 or not
    
    #print('Keeping unique cols')
    if unique_cols==[]:
        unique_cols = keep_unique_cols(tx)
    tx = tx[:,unique_cols]
    len_kept_data = len(unique_cols)
    
    if stdize=='before':
        #print('Standardizing')
        tx = standardize(tx)[0]
    
    print('Cross products')
    tx = add_all_cross_prod(tx)
    
    if stdize=='after':
        #print('Standardizing')
        tx = standardize(tx)[0]
    
    #print('Adding ones')
    tx = add_ones(tx)

    return tx, len_kept_data, unique_cols

In [12]:
def build_poly(x, range_degrees):
    """Polynomial basis functions for input array data x."""
    return np.array([x**p for p in range_degrees]).T 

In [13]:
def add_powers(tx, range_degrees, range_col_idx):
    if len(range_degrees)>0:
        for col_id in range_col_idx:
            tx = np.concatenate((tx, build_poly(tx[:,col_id], range_degrees)), axis=1)
    return tx

In [14]:
def cross_validation_one_fold_LS(y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                 degrees, len_kept_data, stdize=False):
    
    accuracies_train_by_deg = np.zeros(len(degrees))
    accuracies_test_by_deg = np.zeros(len(degrees))
    
    # For each degree, compute the least squares weights, the predictions and the accuracies
    previous_deg = 1
    for deg_id, deg in enumerate(degrees):
        print('++ Degree', deg, '++')
        
        # Add powers of the chosen columns
        tx_cross_val_train = add_powers(tx_cross_val_train, range(previous_deg+1, deg+1), range(len_kept_data))
        tx_cross_val_test = add_powers(tx_cross_val_test, range(previous_deg+1, deg+1), range(len_kept_data))
        if stdize: ### TO BE CHECKED TODOOOOOOOOOOOOO
            tx_cross_val_train = standardize(tx_cross_val_train)[0]
            tx_cross_val_test = standardize(tx_cross_val_test)[0]
    
        # Compute the best weights on the training set
        weights, loss = ridge_regression(y_cross_val_train, tx_cross_val_train, 1e-8, fct='mse')
        #least_squares(y_cross_val_train, tx_cross_val_train, 'mse') !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        # Compute the predictions
        y_predicted_cross_val_train = predict_labels(weights, tx_cross_val_train)
        y_predicted_cross_val_test = predict_labels(weights, tx_cross_val_test)

        # Compute the accuracies for each degree
        accuracies_train_by_deg[deg_id] = \
            np.sum(y_predicted_cross_val_train == y_cross_val_train)/len(y_cross_val_train)
        accuracies_test_by_deg[deg_id] = \
            np.sum(y_predicted_cross_val_test == y_cross_val_test)/len(y_cross_val_test)
        
        # Update the previous degree to the actual degree
        previous_deg = deg
        
    return accuracies_train_by_deg, accuracies_test_by_deg

In [15]:
def cross_validation_least_squares(y_single_jet_train, tx_single_jet_train, degrees, k_fold, seed):
    
    # Get the indices so that we get the k'th subgroup in test, others in train, for each k
    k_indices = build_k_indices(y_single_jet_train, k_fold, seed)
    
    # Initialize matrix of computed accuracies for each degree and each fold
    accuracies_train_by_fold = np.zeros([len(degrees), k_fold])
    accuracies_test_by_fold = np.zeros([len(degrees), k_fold])
    
    # Preprocess training dataset
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'before')
    
    for k in range(k_fold):
        print('--- Fold', k, '---')
        # Create the testing set for this fold number
        k_index = k_indices[k] # Indices of the testing set for fold k
        y_cross_val_test = y_single_jet_train[k_index]
        tx_cross_val_test = tx_single_jet_train_preprocessed[k_index,:]
        
        # Create the training set for this fold number
        mask = np.ones(len(y_single_jet_train), dtype=bool) # set all elements to True
        mask[k_index] = False # set test elements to False
        y_cross_val_train = y_single_jet_train[mask] # select only True elements (ie train elements)
        tx_cross_val_train = tx_single_jet_train_preprocessed[mask,:]    
        
        # Compute the accuracies for each degree
        accuracies_train_by_fold[:,k], accuracies_test_by_fold[:,k] = cross_validation_one_fold_LS\
            (y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, degrees, len_kept_data, \
             False)
    
    # Compute the mean accuracies over the folds, for each degree
    mean_accuracies_train_by_deg = np.mean(accuracies_train_by_fold, axis=1)
    mean_accuracies_test_by_deg = np.mean(accuracies_test_by_fold, axis=1)
    
    # Get the index of the best accuracy in the testing set
    max_id_deg_test = np.unravel_index(mean_accuracies_test_by_deg.argmax(), mean_accuracies_test_by_deg.shape)
    
    # Find the optimal degree and the corresponding accuracies in the training and testing sets
    best_deg = degrees[max_id_deg_test[0]]
    best_accuracy_test = mean_accuracies_test_by_deg[max_id_deg_test[0]]
    corresponding_accuracy_train = mean_accuracies_train_by_deg[max_id_deg_test[0]]
    
    print('Best accuracy test =', best_accuracy_test, 'with degree =', best_deg)
    print('Corresponding accuracy train =', corresponding_accuracy_train)
    
    return best_deg, best_accuracy_test, corresponding_accuracy_train

# Least Squares

In [22]:
degrees = range(1,4) #range(7,11)
k_fold = 4
seed = 1

In [23]:
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)

y_predicted_train = np.zeros(len(y_train))
y_predicted_test = np.zeros(tx_test.shape[0])
best_degrees = np.zeros(len_mask)

In [29]:
for jet_id in range(len_mask):
    print('***** Jet ', jet_id, '*****')
    # SEPARATE THE WHOLE DATA SET TO GET ONLY THE PART THAT HAVE THE RIGHT NUMBER OF JETS
    tx_single_jet_train = tx_train[mask_jets_train[jet_id]]
    tx_single_jet_test = tx_test[mask_jets_test[jet_id]]
    y_single_jet_train = y_train[mask_jets_train[jet_id]]
    
    # CALL CROSS VALIDATION FOR A SINGLE JET ON TRAIN PART, FIND BEST DEG, BEST ACCURACY ON TESTING CROSS VAL
    best_deg, best_accuracy, corresponding_accuracy_train = cross_validation_least_squares(y_single_jet_train, \
                                                                                           tx_single_jet_train, \
                                                                                           degrees, k_fold, seed)
    
    # KEEP IN MEMORY THE BEST DEGREE FOR THIS JET
    best_degrees[jet_id] = best_deg
    
    # PREPROCESS FULL TRAINING AND TESTING DATA
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'before')
    tx_single_jet_test_preprocessed = preprocess_data(tx_single_jet_test, unique_cols, 'before')[0]
    
    # ADD POWERS TO THE CHOSEN COLUMNS
    tx_single_jet_train_preprocessed = add_powers(tx_single_jet_train_preprocessed, range(2,best_deg), \
                                                  range(len_kept_data))
    tx_single_jet_test_preprocessed = add_powers(tx_single_jet_test_preprocessed, range(2,best_deg), \
                                                 range(len_kept_data))
    
    # COMPUTE THE BEST WEIGHTS AND FULL ACCURACY ON TRAINING FULL SET - ONE JET
    weights, loss = ridge_regression(y_single_jet_train, tx_single_jet_train_preprocessed, 1e-8, fct='mse')
    # least_squares(y_single_jet_train, tx_single_jet_train_preprocessed, 'mse') !!!!!!!!!!!!!!!
    
    # COMPUTE THE PREDICTIONS ON THE FULL TESTING SET - SINGLE JET
    y_predicted_single_jet_train = predict_labels(weights, tx_single_jet_train_preprocessed)
    y_predicted_single_jet_test = predict_labels(weights, tx_single_jet_test_preprocessed)
    
    # ADD THE PREDICTIONS TO y_predicted_test AND y_predicted_train
    y_predicted_train[mask_jets_train[jet_id]] = y_predicted_single_jet_train
    y_predicted_test[mask_jets_test[jet_id]] = y_predicted_single_jet_test
    
    # COMPUTE THE ACCURACY train ON JET
    accuracy_train_single_jet = np.sum(y_predicted_single_jet_train == y_single_jet_train)/len(y_single_jet_train)
    
    # PRINT ACCURACY train ON JET
    print('Accuracy full train on jet', jet_id, '=', accuracy_train_single_jet)

***** Jet  0 *****
--- Fold 0 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
--- Fold 1 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
--- Fold 2 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
--- Fold 3 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
Best accuracy test = 0.829539995196 with degree = 3
Corresponding accuracy train = 0.829869219991
Accuracy full train on jet 0 = 0.825768418524
***** Jet  1 *****
--- Fold 0 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
--- Fold 1 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
--- Fold 2 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
--- Fold 3 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
Best accuracy test = 0.765926441762 with degree = 3
Corresponding accuracy train = 0.76665720967
Accuracy full train on jet 1 = 0.756383472609
***** Jet  2 *****
--- Fold 0 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
--- Fold 1 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3 ++
--- Fold 2 ---
++ Degree 1 ++
++ Degree 2 ++
++ Degree 3

In [30]:
# CREATE CSV SUBMISSION
create_csv_submission(ids_test, y_predicted_test, 'output/trial.csv')

# COMPUTE ACCURACY ON FULL train
total_accuracy_train = np.sum(y_predicted_train == y_train)/len(y_train)*100
print('Total accuracy train =', total_accuracy_train, 'with degrees =', best_degrees)

Total accuracy train = 78.8492 with degrees = [ 3.  3.  3.  3.]


In [31]:
np.sum(y_predicted_test==-1)

407480

In [32]:
np.sum(y_predicted_test==1)

160758